# This is the code for Quantum Optimization

In [63]:
import unittest
import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
import numpy as np
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
from qiskit.primitives import Sampler, Estimator
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
import qiskit_aer as Aer
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter
import matplotlib
matplotlib.use('TkAgg')  # Set non-interactive backend
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import os

In [64]:
pip show qiskit

Name: qiskit
Version: 1.2.4
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: c:\users\arthu\miniconda3\envs\quantumqiskit\lib\site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-algorithms, qiskit-ibm-runtime, qiskit-machine-learning, qiskit-optimization
Note: you may need to restart the kernel to use updated packages.


## We first create the Docplex/Cplex optimization problem

In [65]:
opt_model = Model(name="MIP Model")

Nc= 3 # Nc is the number of seconds

Dist=2 # Distance to travel

tolerance = 0 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=2 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

"""
We define two binary variables for two bits. 
When x=0 and y=0 then constant velocity
When x=1 and y=0 then acceleration
When x=0 and y=1 then breaking
"""

x={}
for i in range(0, Nc):
    x[i]= opt_model.binary_var(name=f"x_{i}")

y={}
for i in range(0, Nc):
    y[i]= opt_model.binary_var(name=f"y_{i}")

z={}
for i in range(0, Nc):
    z[i]= opt_model.binary_var(name=f"z_{i}")
    

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize
## Energy:
for i in range(0, Nc):
    objective += (delta_v**2) * x[i] - alpha*(delta_v**2)*y[i]
    #objective += (delta_v**2)*x[i]

    
opt_model.minimize(objective)

In [66]:
## Constraint 1: (simultaneous braking/acceleration)


for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= x[i] , f"z_u_d_{i}") 

for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= y[i], f"z_p_d_{i}")
        
for i in range(0, Nc):
    opt_model.add_constraint(z[i] >= x[i]+ y[i] -1 , f"z_u_p_d_{i}") 

for i in range(0,Nc):
    opt_model.add_constraint(z[i] == 0, "No simultaneous braking or acceleration constraint"+str(i))
'''opt_model.add_constraint(opt_model.sum(z[i] for i in range(0, Nc)) == 0 , "No_simultaneous_braking_or_acceleration_constraint")
'''
#### Print the optimization model info

opt_model.print_information()
opt_model.prettyprint()

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool z_0;
dvar bool z_1;
dvar bool z_2;

minimize
 x_0 + x_1 + x_2 - 0.050000 y_0 - 0.050000 y_1 - 0.050000 y_2;
 
subject to {
 z_u_d_0:
  z_0 <= x_0;
 z_u_d_1:
  z_1 <= x_1;
 z_u_d_2:
  z_2 <= x_2;
 z_p_d_0:
  z_0 <= y_0;
 z_p_d_1:
  z_1 <= y_1;
 z_p_d_2:
  z_2 <= y_2;
 z_u_p_d_0:
  z_0 >= x_0 + y_0 -1;
 z_u_p_d_1:
  z_1 >= x_1 + y_1 -1;
 z_u_p_d_2:
  z_2 >= x_2 + y_2 -1;
 No_simultaneous_braking_or_acceleration_constraint0:
  z_0 == 0;
 No_simultaneous_braking_or_acceleration_constraint1:
  z_1 == 0;
 No_simultaneous_braking_or_acceleration_constraint2:
  z_2 == 0;

}


In [67]:
"""
Constraint 2: (Total Distance constraints)
"""

distance = opt_model.linear_expr()
velocity = 0
for i in range(0, Nc):
    velocity = velocity + delta_v*(x[i]-y[i])
    distance += velocity
'''opt_model.add_constraint(distance <= Dist+tolerance, "Max_Distance_constraint")
opt_model.add_constraint(distance >= Dist-tolerance, "Min_Distance_constraint")'''
opt_model.add_constraint(distance == Dist, "Distance_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [68]:
"""
Constraint 3: (Net-Zero contraint)

"""
opt_model.add_constraint(opt_model.sum((y[i]-x[i]) for i in range(0, Nc)) == 0 , "Net_Zero_constraint")

#### Print the optimization model info

opt_model.print_information()

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 14
   - linear=14
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [69]:
"""
Constraint 4: (Maximum Speed)

"""
opt_model.add_constraint(opt_model.sum((delta_v*(x[i]) for i in range(0, Nc))) <= vmax , "Maximum_Speed_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 15
   - linear=15
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [70]:

#Constraint 5: (Positive Speed)
for i in range(Nc):
    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint"+str(i))

    #### Print the optimization model

opt_model.print_information()


Model: MIP Model
 - number of variables: 9
   - binary=9, integer=0, continuous=0
 - number of constraints: 18
   - linear=18
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [71]:
'''
#Constraint 6: (Must leave immediately)

opt_model.add_constraint(x[0]==1, "Must leave immediately constraint")
    

#### Print the optimization model info

opt_model.print_information()
'''

'\n#Constraint 6: (Must leave immediately)\n\nopt_model.add_constraint(x[0]==1, "Must leave immediately constraint")\n    \n\n#### Print the optimization model info\n\nopt_model.print_information()\n'

In [72]:
#### Print the optimization model
print(opt_model.prettyprint())

// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;
dvar bool z_0;
dvar bool z_1;
dvar bool z_2;

minimize
 x_0 + x_1 + x_2 - 0.050000 y_0 - 0.050000 y_1 - 0.050000 y_2;
 
subject to {
 z_u_d_0:
  z_0 <= x_0;
 z_u_d_1:
  z_1 <= x_1;
 z_u_d_2:
  z_2 <= x_2;
 z_p_d_0:
  z_0 <= y_0;
 z_p_d_1:
  z_1 <= y_1;
 z_p_d_2:
  z_2 <= y_2;
 z_u_p_d_0:
  z_0 >= x_0 + y_0 -1;
 z_u_p_d_1:
  z_1 >= x_1 + y_1 -1;
 z_u_p_d_2:
  z_2 >= x_2 + y_2 -1;
 No_simultaneous_braking_or_acceleration_constraint0:
  z_0 == 0;
 No_simultaneous_braking_or_acceleration_constraint1:
  z_1 == 0;
 No_simultaneous_braking_or_acceleration_constraint2:
  z_2 == 0;
 Distance_constraint:
  3 x_0 - 3 y_0 + 2 x_1 - 2 y_1 + x_2 - y_2 == 2;
 Net_Zero_constraint:
  y_0 - x_0 + y_1 - x_1 + y_2 - x_2 == 0;
 Maximum_Speed_constraint:
  x_0 + x_1 + x_2 <= 2;
 Positive_Speed_constraint0:
  0 >= 0;
 Positive_

## Problem conversion

In [73]:
# Conversion to qad_model
qp_quad = from_docplex_mp(opt_model)
print("Number of variables:", len(qp_quad.variables))

Number of variables: 9


C:\Users\arthu\AppData\Local\Temp\ipykernel_9528\414607200.py:2: UserWarning: Trivial constraint: Positive_Speed_constraint0: 0 >= 0
  qp_quad = from_docplex_mp(opt_model)


In [74]:
'''ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(qp_quad)'''
qp_eq=qp_quad


# Conversion to qubo
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp_eq)

print(f"QUBO variables: {len(qubo.variables)}")
print(f"QUBO constraints: {len(qubo.linear_constraints)}")

'''opt_model.export_as_lp("qubo1.lp")'''

QUBO variables: 19
QUBO constraints: 0


'opt_model.export_as_lp("qubo1.lp")'

In [75]:
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))
num_qubits = qubitOp.num_qubits
print(num_qubits)
print(offset)

Offset: 1407.0624999999986
Ising Hamiltonian:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIZIIIIIIII', 'IIIIIIIIIZIIIIIIIII', 'IIIIIIIIZIIIIIIIIII', 'IIIIIIIZIIIIIIIIIII', 'IIIIIIZIIIIIIIIIIII', 'IIIIIZIIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIIZIIIIZ', 'IIIIIIIIIIIIZIIIIIZ', 'IIIIIIIIIZIIIIIIIIZ', 'IIIIIIIIZIIIIIIIIIZ', 'IIIZIIIIIIIIIIIIIIZ', 'IIZIIIIIIIIIIIIIIIZ', 'IZIIIIIIIIIIIIIIIIZ', 'IZIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIIIZ', 'ZIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIIIZIIIZI', 'IIIIIIIIIIIZIIIIIZI', 'IIIIIIIZIIIIIIIIIZI', 'IIIIIIZIIIIIIIIIIZI', 'IIIZIIIIIIIIIIIIIZI', 'IIZIIIIIIIIIIIIIIZI', 'IZIIIIIIIIIIIIIIIZ

In [76]:
p=4
qubo_circuit = QAOAAnsatz(cost_operator=qubitOp, reps=p)
qubo_circuit.measure_all()
print(qubo_circuit.num_qubits)

19


In [77]:
qubo_circuit.draw( style={'backgroundcolor': '#FFFFFF'})

┌─────────────────────────────────────────────────┐ ░ ┌─┐            »
    q_0: ┤0                                                ├─░─┤M├────────────»
         │                                                 │ ░ └╥┘┌─┐         »
    q_1: ┤1                                                ├─░──╫─┤M├─────────»
         │                                                 │ ░  ║ └╥┘┌─┐      »
    q_2: ┤2                                                ├─░──╫──╫─┤M├──────»
         │                                                 │ ░  ║  ║ └╥┘┌─┐   »
    q_3: ┤3                                                ├─░──╫──╫──╫─┤M├───»
         │                                                 │ ░  ║  ║  ║ └╥┘┌─┐»
    q_4: ┤4                                                ├─░──╫──╫──╫──╫─┤M├»
         │                                                 │ ░  ║  ║  ║  ║ └╥┘»
    q_5: ┤5                                                ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
    q_6: ┤6                                                ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
    q_7: ┤7                                                ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
    q_8: ┤8                                                ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
    q_9: ┤9  QAOA(γ[0],β[0],γ[1],β[1],γ[2],β[2],γ[3],β[3]) ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_10: ┤10                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_11: ┤11                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_12: ┤12                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_13: ┤13                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_14: ┤14                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_15: ┤15                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_16: ┤16                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_17: ┤17                                               ├─░──╫──╫──╫──╫──╫─»
         │                                                 │ ░  ║  ║  ║  ║  ║ »
   q_18: ┤18                                               ├─░──╫──╫──╫──╫──╫─»
         └─────────────────────────────────────────────────┘ ░  ║  ║  ║  ║  ║ »
meas: 19/═══════════════════════════════════════════════════════╩══╩══╩══╩══╩═»
                                                                0  1  2  3  4 »
«                                                   
«    q_0: ──────────────────────────────────────────
«                                                   
«    q_1: ──────────────────────────────────────────
«                                                   
«    q_2: ──────────────────────────────────────────
«                                                   
«    q_3: ──────────────────────────────────────────
«                                                   
«    q_4: ──────────────────────────────────────────
«         ┌─┐                                       
«    q_5: ┤M├───────────────────────────────────────
«         └╥┘┌─┐                                    
«    q_6: ─╫─┤M├────────────────────────

In [78]:
'''expanded_circuit = qubo_circuit.decompose()
expanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})'''

"expanded_circuit = qubo_circuit.decompose()\nexpanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})"

In [79]:
qubo_circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(β[2]), ParameterVectorElement(β[3]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1]), ParameterVectorElement(γ[2]), ParameterVectorElement(γ[3])])

### Optimize circuit

In [80]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>", overwrite=True, set_as_default=True)
'''service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.least_busy(min_num_qubits=num_qubits+5)'''
backend = AerSimulator()
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)

candidate_circuit = pm.run(qubo_circuit)
'''candidate_circuit.draw()'''
num_qubits_circ = candidate_circuit.num_qubits
print(num_qubits_circ)

AerSimulator('aer_simulator')
19


### Execute circuit

In [81]:
initial_gamma = np.pi
initial_beta = np.pi/2
init_params = []
for i in range(p):
    init_params.append(initial_gamma)
    init_params.append(initial_beta)

In [82]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs
    total_cost = cost + offset

    objective_func_vals.append(total_cost)


    return total_cost

In [83]:
objective_func_vals = [] # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000

    # Set simple error suppression/mitigation options, this is for when on quantum hardware
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, qubitOp, estimator),
        method="COBYLA",
        tol=1e-3,
    )
    print(result)

c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")
c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")
c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': 

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 1097.3084549999987
       x: [ 2.463e+00  1.768e+00  3.582e+00  1.508e+00  4.145e+00
            1.576e+00  3.598e+00  1.313e+00]
    nfev: 68
   maxcv: 0.0


In [84]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [85]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)

In [86]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {key: val/shots for key, val in counts_int.items()}

final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)
print(final_distribution_bin)

{36259: 0.0001, 14623: 0.0001, 140683: 0.0001, 460227: 0.0002, 498147: 0.0002, 287903: 0.0001, 18771: 0.0001, 84338: 0.0001, 171511: 0.0001, 411852: 0.0001, 164291: 0.0001, 68818: 0.0001, 206735: 0.0001, 451063: 0.0001, 332685: 0.0002, 332935: 0.0001, 442071: 0.0001, 480997: 0.0001, 445199: 0.0001, 96381: 0.0002, 92637: 0.0001, 28436: 0.0001, 33230: 0.0005, 83695: 0.0001, 44349: 0.0001, 468068: 0.0001, 269798: 0.0001, 493391: 0.0001, 223047: 0.0001, 461505: 0.0001, 483819: 0.0001, 395719: 0.0004, 133095: 0.0001, 164019: 0.0001, 304643: 0.0001, 434415: 0.0001, 460189: 0.0001, 344619: 0.0001, 25925: 0.0002, 265939: 0.0001, 296911: 0.0005, 87248: 0.0001, 123729: 0.0001, 364663: 0.0001, 263618: 0.0004, 27500: 0.0001, 60366: 0.0001, 1349: 0.0002, 281064: 0.0001, 314781: 0.0001, 411540: 0.0001, 228255: 0.0001, 254594: 0.0001, 353889: 0.0001, 360151: 0.0001, 358727: 0.0002, 140738: 0.0001, 295303: 0.0002, 22729: 0.0001, 33226: 0.0002, 68079: 0.0001, 61190: 0.0001, 319692: 0.0001, 289141: 0.00

c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [87]:
# Sort both distributions by probability in descending order
sorted_dist_int = dict(sorted(final_distribution_int.items(), key=lambda x: x[1], reverse=True))
sorted_dist_bin = dict(sorted(final_distribution_bin.items(), key=lambda x: x[1], reverse=True))

# Print top 10 most likely outcomes
print("\nTop 10 outcomes (binary representation):")
for k, v in list(sorted_dist_bin.items())[:5]:
    print(f"{k}: {v}")


Top 10 outcomes (binary representation):
0000000000111001110: 0.0009
1100000000111000111: 0.0007
0001000010110001111: 0.0006
1100000000111001110: 0.0006
0001000000111001110: 0.0005


### Post-process

In [88]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]

keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, num_qubits)
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [89]:
x_value = most_likely_bitstring[0:Nc]
y_value = most_likely_bitstring[Nc:2*Nc]

print(x_value)
print(y_value)

[0, 1, 1]
[1, 0, 0]


### Visualization

In [90]:
def distance(x, y, Nc):
    velocity = 0
    vel = [0]
    dist = [0]
    dist_tot= 0
    for i in range(0, Nc):
        velocity = velocity + delta_v*(x[i]-y[i])
        if velocity < 0:
            velocity = -velocity
        vel.append(velocity)
        dist_tot += velocity
        dist.append(dist_tot)
    return dist,vel

In [91]:
time= np.arange(Nc+1)
distn, velo = distance(x_value, y_value, Nc)

In [92]:
plt.figure(figsize=(15,8))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, velo, c='b', marker="o", markersize=2, linestyle='-')#, label='label)

plt.xlabel('Time')
plt.ylabel('Velocity')
plt.title("Velocity vs Time")
plt.grid(axis='x')
plt.grid(axis='y')
plt.legend()
plt.show()

C:\Users\arthu\AppData\Local\Temp\ipykernel_9528\2353725290.py:12: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [93]:
plt.figure(figsize=(10,6))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, distn, c='b', marker="o", markersize=1, linestyle='-')#, label='label')
plt.xlabel('Time')
plt.ylabel('Distance')
plt.title("Distance vs Time")
plt.legend()
plt.show()

C:\Users\arthu\AppData\Local\Temp\ipykernel_9528\1348846773.py:9: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
